In [29]:
from sklearn.model_selection import train_test_split
import pandas as pd

In [30]:
from PIL import Image
import matplotlib.pyplot as plt
# import seaborn as sns

import pytesseract
import re

In [31]:
from nltk.corpus import stopwords
# NLTK (Natural Language Toolkit) 
# conjunto de palavras stopwords para o idioma inglês 
# Lista de palavras que são consideradas stopwords (palavras vazias) para o idioma inglês. 
stopwords_list = stopwords.words("english")

In [32]:
def preprocess_data(text):
    text = text.lower()
    text = text.replace("\n"," ").replace("\t"," ")
    text = re.sub("\s+"," ",text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    
    # tokens
    tokens = word_tokenize(text)
    
    data = [i for i in tokens if i not in punctuation]
    data = [i for i in data if i not in stopwords_list]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    final_text = []
    for i in data:
        word = lemmatizer.lemmatize(i)
        final_text.append(word)
        
    return " ".join(final_text)

In [33]:
import os
import pyarrow.parquet as pq

# Definição de Classes, opções de target
class_labels = {'email':0,'resume':1,'scientific_publication':2}

# usada para armazenar os textos extraídos das imagens após o pré-processamento
final_text = []
# usada para armazenar os rótulos numéricos correspondentes a cada documento
final_label = []
path = "dataset/www-kaggle-com_datasets_ritvik1909_document-classification-dataset"
image_folder = os.listdir(path)

# Verificar se o arquivo Parquet já existe, para não precisar reprocessar as imagem novamente
parquet_file = "document_texts.parquet"
if os.path.exists(parquet_file):
    # Carregar dados existentes se o arquivo Parquet já existir
    df = pd.read_parquet(parquet_file)
    final_text = df['Text'].tolist()
    final_label = df['Label'].tolist()
else:
    # para cada imagem do dataset
    for label_dir in image_folder:
        # caminho completo para o diretório atual de documentos (por classe 'email', 'resume' e 'scientific_publication')
        label_path = os.path.join(path,label_dir)
        # Itera sobre os arquivos dentro do diretório ""
        for j in os.listdir(label_path):
            # carrega a imagem.png
            image = Image.open(label_path+"/"+j)
            # Usa Tesseract OCR para extrair o texto da imagem
            text = pytesseract.image_to_string(image)
            # executa preprocessamento
            text_data = preprocess_data(text)
            # Adiciona o texto pré-processado (text_data) à lista final_text
            final_text.append(text_data)
            # Adiciona o rótulo numérico correspondente ao tipo de documento (label) à lista final_label, usando o dicionário class_labels para fazer a correspondência entre nome do diretório e rótulo numérico.
            final_label.append(class_labels[label_dir])
            
    # Criar um DataFrame com os resultados
    df = pd.DataFrame({'Text': final_text, 'Label': final_label})
    # Salvar DataFrame em arquivo Parquet
    df.to_parquet(parquet_file, index=False)

In [34]:
# Criando Dataframe pandas com base nas lista 'final_text' e 'final_label'
# Coluna 'Text' representa o texto
# Coluna 'Label' tem valores 0, 1 e 2 (classes 'email', 'resume' e 'scientific_publication')
df = pd.DataFrame()
df['Text'] = final_text
df['Label'] = final_label
df['Label']

0      2
1      2
2      2
3      2
4      2
      ..
160    0
161    0
162    0
163    0
164    0
Name: Label, Length: 165, dtype: int64

In [35]:
# Dividindo o dataframe em treino e teste (split)
x_train,x_test,y_train,y_test = train_test_split(df['Text'],df['Label'],shuffle=True,test_size=0.10)


In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Configurando o TfidfVectorizer
tfidf = TfidfVectorizer(ngram_range=(2,5), max_df=0.95, min_df=2 ,max_features=10000)

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier


# Vetorizando os documentos
tfidf = TfidfVectorizer(ngram_range=(2, 5), max_df=0.95, min_df=2, max_features=10000)
tfidf_train = tfidf.fit_transform(df['Text'])

# Separando os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(tfidf_train, df['Label'], test_size=0.2, random_state=42)

# Lista de modelos para comparar
models = {
    'Logistic Regression': LogisticRegression(),
    'Random Forest': RandomForestClassifier(),
    'SVM': SVC(),
    'Naive Bayes': MultinomialNB(),
    'KNN': KNeighborsClassifier(),
    'Gradient Boosting': GradientBoostingClassifier()
}

# Dicionário para armazenar as acurácias
accuracies = {}

# Treinando e avaliando cada modelo
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    accuracies[name] = accuracy

print("\nAcurácias dos modelos:")
for name, accuracy in accuracies.items():
    print(f'{name}: {accuracy:.4f}')



Acurácias dos modelos:
Logistic Regression: 0.9394
Random Forest: 0.9394
SVM: 0.9091
Naive Bayes: 1.0000
KNN: 0.4242
Gradient Boosting: 0.8485
